In [1]:
import gurobipy as gp
import numpy as np
import ciropt as co
import sympy as sp

In [2]:
L_smooth = 1.
mu = 0.0001
Capacitance = 1.

time_limit = 500
ps = 10
psm = 1

solver = "gp"

In [3]:
def gp_print_solutions(model, gp_vars):
    for i in range(model.SolCount):
        model.Params.SolutionNumber = i
        # print(ns, gp_vars)
        print(f"{i+1}: obj = {model.PoolObjVal}")
        print(model.printQuality())
        # print(gp_vars["b"].Xn, gp_vars["h"].Xn,  gp_vars["alpha"].Xn,  gp_vars["beta"].Xn)
        print(co.dict_parameters_ciropt_gp(model, gp_vars, all=True, Xn=True))
    model.Params.SolutionNumber = 0

In [4]:
problem = co.gradient_flow_circuit(mu, L_smooth, Capacitance)
problem.obj = problem.b
res, co_model, sp_exp = problem.solve(solver=solver, verbose=False, debug=True, time_limit=time_limit, psm=psm, ps=ps)
gp_vars = problem.vars
print(res)

dim_G=5, dim_F=4
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-31
{'b': 0.0, 'h': 0.0, 'd': 0.0, 'alpha': -0.25, 'beta': -0.5}


In [5]:
co_vars = co.dict_parameters_ciropt_gp(co_model, gp_vars, all=True)

In [6]:
co_vars

{'b': 0.0,
 'd': 0.0,
 'h': 0.0,
 'gamma': 0.0,
 'alpha': -0.25,
 'beta': -0.5,
 'P': array([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]),
 'lamb0': array([[-0.,  0.,  0.,  0.],
        [ 0., -0.,  0.,  0.],
        [ 0.,  0., -0.,  0.],
        [ 0.,  0.,  0., -0.]]),
 'alpha_h': 0.0,
 'beta_h': 0.0,
 'alpha_alpha': 0.0625,
 'alpha_alpha_h': 0.0,
 'alpha_alpha_h_h': 0.0,
 'alpha_beta': 0.125,
 'alpha_beta_h': 0.0,
 'alpha_beta_h_h': 0.0,
 'beta_beta': 0.25,
 'beta_beta_h': 0.0,
 'beta_beta_h_h': 0.0,
 'alpha_h_h': 0.0,
 'beta_h_h': 0.0,
 'h_h': 0.0}

In [7]:
# gp_print_solutions(problem.model, gp_vars)

In [8]:
size_I = 4          # I = { star, 1, 1.5, 2 }
dim_G = 5           # { x_star, x_P_1, g_1, g_1p5, g_2 }
dim_F = 4           # { f_star, f_1, f_1p5, f_2 }
model = gp.Model()

b = model.addVar(name='b', lb=0., ub=2)
h = model.addVar(name='h', lb=0., ub=10)
alpha = model.addVar(name='alpha', lb=-100, ub=100) #lb=-1.*gp.GRB.INFINITY, ub=gp.GRB.INFINITY)
beta = model.addVar(name='beta', lb=-100, ub=100) #lb=-1.*gp.GRB.INFINITY, ub=gp.GRB.INFINITY)
alpha_h = model.addVar(name='alpha_h', lb=-1000, ub=1000)
beta_h = model.addVar(name='beta_h', lb=-1000, ub=1000)
P = model.addMVar((dim_G, dim_G), name='P', lb=-1.*gp.GRB.INFINITY, ub=gp.GRB.INFINITY)
lamb0 = model.addMVar((size_I, size_I), name='lamb0', lb=0, ub=gp.GRB.INFINITY)
model.update()

gram_gp_vars = {'b': b,
                'h': h,
                'alpha': alpha,
                'beta': beta, 
                'P': P, 
                'lamb0': lamb0}

g_star = np.zeros(dim_G).reshape(-1, 1)
g_1 = co.one_hot(dim_G, 2)
g_1p5 = co.one_hot(dim_G, 3) 
g_2 = co.one_hot(dim_G, 4) 
Gs = [g_star, g_1, g_1p5, g_2]

f_star = co.one_hot(dim_F, 0) # np.zeros((dim_F, 1))
f_1   = co.one_hot(dim_F, 1) 
f_1p5 = co.one_hot(dim_F, 2)
f_2   = co.one_hot(dim_F, 3)
F = [f_star, f_1, f_1p5, f_2]

x_P_star = co.one_hot(dim_G, 0)
x_P_1    = co.one_hot(dim_G, 1)
x_P_1p5 = x_P_1 - (alpha_h / Capacitance) * g_1
x_P_2   = x_P_1 - (beta_h / Capacitance) * g_1 - ((h - beta_h) / Capacitance) * g_1p5 
Xs = [x_P_star, x_P_1, x_P_1p5, x_P_2]

sp_vars = {name : sp.symbols(name) for name in gp_vars.keys()}
sp_x_P_1p5 = x_P_1 - (sp_vars["alpha_h"] / Capacitance) * g_1
sp_x_P_2   = x_P_1 - (sp_vars["beta_h"] / Capacitance) * g_1 - ((sp_vars["h"] - sp_vars["beta_h"]) / Capacitance) * g_1p5 
sp_Xs = [x_P_star, x_P_1, sp_x_P_1p5, sp_x_P_2]

star_idx = 4
A = lambda i,j: co.symm_prod(Gs[j], Xs[i] - Xs[j])
B = lambda i,j: co.symm_prod(Xs[i] - Xs[j], Xs[i] - Xs[j])
C = lambda i,j: co.symm_prod(Gs[i] - Gs[j], Gs[i] - Gs[j])
# D = lambda i,j: co.symm_prod(Xs[i] - Xs[j] - (1/L_smooth) * (Gs[i] - Gs[j]))
a = lambda i,j: F[j] - F[i]

sp_A = lambda i,j: co.symm_prod(Gs[j], sp_Xs[i] - sp_Xs[j])
sp_B = lambda i,j: co.symm_prod(sp_Xs[i] - sp_Xs[j], sp_Xs[i] - sp_Xs[j])
sp_C = lambda i,j: co.symm_prod(Gs[i] - Gs[j], Gs[i] - Gs[j])
sp_D = lambda i,j: co.symm_prod(sp_Xs[i] - sp_Xs[j] - (1/L_smooth) * (Gs[i] - Gs[j]))

D = lambda i,j: co.sympy_matrix_to_gurobi(co.simplify_matrix(sp_D(i, j)), gram_gp_vars, model)
assert (dim_G, dim_G) == A(1, 2).shape == B(1, 2).shape == C(1, 2).shape
assert a(1, 2).shape == (dim_F, 1)

model.addConstr( alpha_h == alpha * h )
model.addConstr( beta_h == beta * h )
model.addConstr( lamb0.diagonal() == np.zeros(size_I) )
model.addConstr( P.diagonal() >= np.zeros(dim_G) )
for i in range(dim_G):
            # model.addConstr( P[i, i] >= 0 )
            for j in range(i+1, dim_G):
                model.addConstr( P[i, j] == 0 )

model.setObjective( -b )
# star:0, 1:1, 1p5:2, 2:3
sum_ij_La = np.zeros((dim_F, 1), dtype=object)
sum_ij_AC  = np.zeros((dim_G, dim_G), dtype=object)
for i in range(size_I):
    for j in range(size_I):
        if i == j: continue
        sum_ij_La += lamb0[i, j].item() * a(i, j)
        sum_ij_AC += lamb0[i, j].item() * (A(i, j) + (1./(2 * L_smooth)) * C(i, j) + mu / (2 * (1 - mu / L_smooth)) * D(i, j))

Fweights_d = - b * a(1, 0)
Gweights_d = (Capacitance / 2) * (B(3, 0) - B(1, 0))
# model.addConstr( sum_ij_La == Fweights_d ) 
# print(sum_ij_La, "\n", Fweights_d)
# model.addConstr( sum_ij_AC  - P @ P.T - Gweights_d == np.zeros((dim_G, dim_G)))
z1 = sum_ij_La - Fweights_d
z2 = sum_ij_AC - Gweights_d
PPt = P @ P.T
model.addConstrs(sum_ij_La[i, 0] - Fweights_d[i, 0] == 0 for i in range(dim_F))
model.addConstrs(sum_ij_AC[i, j] - Gweights_d[i, j] - PPt[i, j].item() == 0 for i in range(dim_G) for j in range(dim_G))

model.params.NonConvex = 2
model.params.TimeLimit = time_limit
model.Params.PoolSearchMode = 1
model.Params.FeasibilityTol = 1e-9
model.Params.Method = 0
model.Params.PoolSearchMode = 1
model.Params.Heuristics = 0.001
model.Params.LogToConsole = 0
model.update()
model.optimize()

Set parameter NonConvex to value 2
Set parameter TimeLimit to value 500
Set parameter PoolSearchMode to value 1
Set parameter FeasibilityTol to value 1e-09
Set parameter Method to value 0
Set parameter Heuristics to value 0.001


In [9]:
gram_gp_vars.keys()

dict_keys(['b', 'h', 'alpha', 'beta', 'P', 'lamb0', 'alpha_h', 'alpha_alpha', 'alpha_alpha_h', 'alpha_alpha_h_h', 'beta_h', 'beta_beta', 'beta_beta_h', 'beta_beta_h_h', 'beta_h_h', 'h_h', 'alpha_beta', 'alpha_beta_h', 'alpha_beta_h_h', 'alpha_h_h'])

In [10]:
sol = co.dict_parameters_ciropt_gp(model, gram_gp_vars, all=True)
sol

{'b': 0.9373437500000001,
 'h': 1.25,
 'alpha': -75.0,
 'beta': 0.9999999999999998,
 'P': array([[ 0.00883884,  0.        ,  0.        ,  0.        ,  0.        ],
        [-0.00883883,  0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ]]),
 'lamb0': array([[-0.        ,  1.24984375,  0.        ,  0.        ],
        [ 0.3125    , -0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        , -0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , -0.        ]]),
 'alpha_h': -93.75,
 'alpha_alpha': 5625.0,
 'alpha_alpha_h': 7031.25,
 'alpha_alpha_h_h': 8789.0625,
 'beta_h': 1.25,
 'beta_beta': 1.0000000000009281,
 'beta_beta_h': 1.2500000000011602,
 'beta_beta_h_h': 1.5625000000014502,
 'beta_h_h': 1.5625,
 'h_h': 1.5625,
 'alpha_bet

In [11]:
# gp_print_solutions(model, gram_gp_vars)

In [12]:
# compare matrices
size_I_function = size_I
for i in range(size_I_function):
    for j in range(size_I_function):
        if i == j: continue
        F1, G1 = sp_exp[(0,i,j)]["F"], sp_exp[(0,i,j)]["G"]
        F2 = a(i, j)
        G2 = co.simplify_matrix(sp_A(i, j) + (1./(2*L_smooth)) * sp_C(i, j) + mu / (2 * (1 - mu / L_smooth)) * sp_D(i, j))
        assert co.equal_sp_arrays(G1, G2), print(f"{i=}, {j=} \n{G1=} \n{G2=}")
        assert co.equal_sp_arrays(F1, F2[:, 0]), print(f"{i=}, {j=} \n{F1=} \n{F2=}\n")

F1, G1 = sp_exp["FG_d"]["F"], sp_exp["FG_d"]["G"]
F2 = -sp_vars["b"] * a(1, 0)
G2 = (Capacitance / 2) * (sp_B(3, 0) - sp_B(1, 0))
assert co.equal_sp_arrays(G1, G2), print(f"{G1=} \n{G2=}")
assert co.equal_sp_arrays(F1, F2[:,0]), print(f"{F1=} \n{F2=}\n")

In [13]:
problem.model.printQuality()

In [14]:
h_init = co_vars["h"]
b_init = co_vars["b"]
alpha_init = co_vars["alpha"]
beta_init = co_vars["beta"]
lamb_init = co_vars["lamb0"]
P_init = co_vars["P"]
alpha_h_init = alpha_init * h_init
beta_h_init = beta_init * h_init

In [15]:
model.NumVars, model.objVal

(61, -0.9373437500000001)

In [16]:
gp_vars.keys()

dict_keys(['b', 'd', 'h', 'gamma', 'alpha', 'beta', 'P', 'lamb0', 'alpha_h', 'beta_h', 'alpha_alpha', 'alpha_alpha_h', 'alpha_alpha_h_h', 'alpha_beta', 'alpha_beta_h', 'alpha_beta_h_h', 'beta_beta', 'beta_beta_h', 'beta_beta_h_h', 'alpha_h_h', 'beta_h_h', 'h_h'])

In [17]:
vars = [b, h, alpha, beta, alpha_h, beta_h, P, lamb0]
eval_vars = [b_init, h_init, alpha_init, beta_init, alpha_h_init, beta_h_init, P_init, lamb_init]

for v, v_val in zip(vars, eval_vars):
    v.setAttr(gp.GRB.Attr.LB, v_val) 
    v.setAttr(gp.GRB.Attr.UB, v_val)
model.update()
model.optimize()

In [18]:
b.X, h.X, alpha.X, beta.X

(0.0, 0.0, -0.25, -0.5)

In [20]:
for name in ['h', 'b', 'alpha', 'beta', 'P', 'lamb0']:
    v_eval = globals()[name].X
    assert np.allclose(v_eval, co_vars[name]), print(name, v_eval, co_vars[name])

for name in ['sum_ij_AC', 'sum_ij_La', 'Fweights_d', 'Gweights_d']:
    v_eval = co.gurobi_to_numpy(globals()[name])
    assert np.allclose(0, np.linalg.norm(v_eval.flatten()-\
            co.gurobi_to_numpy(problem.gp_expressions[name]).flatten())), print(name)

sum1 = co.gurobi_to_numpy(sum_ij_La - Fweights_d)
sum2 = co.gurobi_to_numpy(sum_ij_AC - Gweights_d) - co.gurobi_to_numpy(PPt)

assert np.allclose(0, np.linalg.norm(sum1.flatten())) and np.allclose(0, np.linalg.norm(sum2.flatten()))

print("PASSED")

PASSED
